## Exercise 10 | data in f1ex10 | calculate, for each Freguesia of Coimbra, the weighted average travel time to the nearest hospital, considering the resident population. Use the following formula with attributes from the Feature Class”

In [ ]:
units = '/mnt/d/cwsig/exercicios_2023_24/f1ex10/freg_cmb.shp'

units_pk = 'idfreg'

dist = '/mnt/d/cwsig/exercicios_2023_24/f1ex10/dist_hospitais.shp'

units_fk = 'idfreg'

distcol = 'wthsp'
popcol = 'popres'

outcol = 'wdist'

In [ ]:
import pandas as pd

from glass.rd.shp import shp_to_obj

In [ ]:
units_df = shp_to_obj(units)
dist_df  = shp_to_obj(dist)

In [ ]:
dist_df['prod'] = dist_df[distcol] * dist_df[popcol]

prodsum = pd.DataFrame({
    'prodsum' : dist_df.groupby([units_fk])['prod'].agg('sum')
}).reset_index()

prodsum.rename(columns={units_fk : f'{units_fk}_tmp'}, inplace=True)

dist_df = dist_df.merge(
    prodsum, how='left',
    left_on=units_fk, right_on=f'{units_fk}_tmp'
)

dist_df['wcalc'] = dist_df['prod'] / dist_df['prodsum'] * dist_df[distcol]

res = pd.DataFrame({
    outcol : dist_df.groupby([units_fk])['wcalc'].agg('sum')
}).reset_index()

if units_pk == units_fk:
    _units_fk = f'{units_fk}_fk'
    res.rename(columns={units_fk : _units_fk}, inplace=True)

else:
    _units_fk = units_fk

units_df = units_df.merge(res, how='left', left_on=units_pk, right_on=_units_fk)

In [ ]:
units_df